## Tests of connection to the Neo4j database, and basic NeoAccess library operations
### using the `debug` option

In [1]:
import set_path      # Importing this module will add the project's home directory to sys.path

Added 'D:\Docs\- MY CODE\Brain Annex\BA-Win7' to sys.path


In [2]:
import os
import sys
import getpass

from neoaccess import NeoAccess

In [3]:
print("To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )\n")

host = input("Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012 ")
host += ":7687"    # EXAMPLE of host value:  "bolt://123.456.789.012:7687"

password = getpass.getpass("Enter the database password:")

print(f"\n=> Will be using: host='{host}', username='neo4j', password=**********")

To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )



Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012  bolt://155.248.202.124
Enter the database password: ········



=> Will be using: host='bolt://155.248.202.124:7687', username='neo4j', password=**********


In [4]:
db = NeoAccess(host=host,
                credentials=("neo4j", password), debug=True)

~~~~~~~~~ Initializing NeoAccess object ~~~~~~~~~
Attempting to connect to Neo4j host 'bolt://155.248.202.124:7687', with username 'neo4j'
Connection to host 'bolt://155.248.202.124:7687' established


In [5]:
print("Version of the Neo4j driver: ", db.version())

Version of the Neo4j driver:  4.4.11


In [6]:
db.empty_dbase()       # WARNING: USE WITH CAUTION!!!

In [7]:
neo_car = db.create_node("Car", {'color': 'white', 'make': 'Toyota'})
neo_person = db.create_node("Person", {'name': 'Julian'})


In create_node().  Query:
    CREATE (n :`Car` {`color`: $par_1, `make`: $par_2}) RETURN n
Data binding:
    {'par_1': 'white', 'par_2': 'Toyota'}


In create_node().  Query:
    CREATE (n :`Person` {`name`: $par_1}) RETURN n
Data binding:
    {'par_1': 'Julian'}



In [8]:
neo_car

21

In [9]:
neo_person

0

In [10]:
number_added = db.add_links(neo_car, neo_person, rel_name="OWNED_BY")

number_added

In add_links()
    cypher_match_from: CYPHER-PROCESSED match structure:
    node: (from)    where: id(from) = 21    data_binding: {}    dummy_node_name: from
    cypher_match_to: CYPHER-PROCESSED match structure:
    node: (to)    where: id(to) = 0    data_binding: {}    dummy_node_name: to

In add_links().  Query:
    
            MATCH (from), (to)
            WHERE (id(from) = 21 AND id(to) = 0)
            MERGE (from) -[:`OWNED_BY`]-> (to)           
            

In update_query(). Attributes of ResultSummary object:
    metadata -> {'query': '\n            MATCH (from), (to)\n            WHERE (id(from) = 21 AND id(to) = 0)\n            MERGE (from) -[:`OWNED_BY`]-> (to)           \n            ', 'parameters': {}, 'server': <neo4j.api.ServerInfo object at 0x000000001A051F70>, 't_first': 55, 'fields': [], 'bookmark': 'FB:kcwQxosFA855RdybwjuMf2O2J8oAAPzLkA==', 'stats': {'relationships-created': 1}, 'type': 'w', 't_last': 0, 'db': 'neo4j', 'notifications': [{'severity': 'WARNING',

1

In [11]:
match_from = db.match(internal_id=neo_car)
match_to = db.match(internal_id=neo_person)

In [12]:
match_from

In [13]:
match_to

In [14]:
# Retrieve the car nodes
db.get_nodes(neo_car)

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 21    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 21) RETURN n



[{'color': 'white', 'make': 'Toyota'}]

In [15]:
# Retrieve a single property of the car node (for situation when only 1 node is present)
db.get_nodes(neo_car, single_cell="color")

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 21    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 21) RETURN n



'white'

In [16]:
# How many owners does the car have?
db.count_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

In count_links()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 21    data_binding: {}    dummy_node_name: n

In count_links().  Query:
    MATCH (n) - [:OWNED_BY] -> (neighbor )WHERE (id(n) = 21) RETURN count(neighbor) AS link_count



1

In [17]:
# Look up information about the car owner(s)
db.follow_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

In follow_links()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 21    data_binding: {}    dummy_node_name: n

In follow_links().  Query:
    MATCH (n) - [:OWNED_BY] -> (neighbor )WHERE (id(n) = 21) RETURN neighbor



[{'name': 'Julian'}]

#### 'Julian' is indeed being retrieved as the owner of the car...  :)